Selección de Atributos
======================

No todos los atributos de un conjunto de datos son útiles al momento de entrenar un modelo de aprendizaje automatizado. En especial en algoritmos paramétricos, como los lineales, algunos atributos pueden ser redundantes o incluso perjudiciales para el desempeño del modelo. Por ejemplo, en un modelo de regresión lineal, si dos atributos están altamente correlacionados, el modelo puede tener problemas para encontrar los parámetros óptimos. Por otro lado, si un atributo no tiene correlación con la variable objetivo, el modelo puede tener un desempeño pobre. Debido a esto, una tarea importante a realizar durante la etapa de preprocesamiento de datos, es la selección de atributos.

Esta selección se puede realizar de forma manual, en base a conocimiento experto, o de forma automática, utilizando algoritmos de selección de atributos. Dentro de los métodos automaticos, uno puede realizar una optimización de la selección basada en probar combinaciones de atributos, quedandose con aquellos que mejor desempeño entreguen. Otro método es analizar por medio de una matriz de correlación la relación entre los atributos, eliminando aquellos que estén altamente correlacionados. Finalmente, se pueden utilizar algoritmos de selección de atributos, que calculan pesos para cada atributo, y se quedan con aquellos que tengan mayor peso o superen un umbral.

## Atributos Correlacionados

### Python

Carguemos un conjunto de datos y veamos como se comportan los atributos entre si.

In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel

In [35]:
dataset: pd.DataFrame = pd.read_csv('./datasets/full.csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899 entries, 0 to 898
Data columns (total 77 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   dataset   899 non-null    object 
 1   id        899 non-null    int64  
 2   ccf       899 non-null    int64  
 3   age       899 non-null    int64  
 4   sex       899 non-null    int64  
 5   painloc   617 non-null    float64
 6   painexer  617 non-null    float64
 7   relrest   613 non-null    float64
 8   pncaden   0 non-null      float64
 9   cp        899 non-null    int64  
 10  trestbps  840 non-null    float64
 11  htn       865 non-null    float64
 12  chol      869 non-null    float64
 13  smoke     230 non-null    float64
 14  cigs      479 non-null    float64
 15  years     467 non-null    float64
 16  fbs       809 non-null    float64
 17  dm        95 non-null     float64
 18  famhist   477 non-null    float64
 19  restecg   897 non-null    float64
 20  ekgmo     846 non-null    float6

In [36]:
correlation_matrix = dataset.drop(['num', 'ccf', 'name', 'dataset'], axis=1).corr().abs()

upper = correlation_matrix.where(
    np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
)

print(upper)

          id       age       sex   painloc  painexer   relrest  pncaden  \
id       NaN  0.098213  0.067022  0.029687  0.135675  0.004992      NaN   
age      NaN       NaN  0.063616  0.012959  0.193591  0.198020      NaN   
sex      NaN       NaN       NaN  0.103689  0.222428  0.258928      NaN   
painloc  NaN       NaN       NaN       NaN  0.269258  0.332300      NaN   
painexer NaN       NaN       NaN       NaN       NaN  0.694375      NaN   
...       ..       ...       ...       ...       ...       ...      ...   
lvx3     NaN       NaN       NaN       NaN       NaN       NaN      NaN   
lvx4     NaN       NaN       NaN       NaN       NaN       NaN      NaN   
lvf      NaN       NaN       NaN       NaN       NaN       NaN      NaN   
cathef   NaN       NaN       NaN       NaN       NaN       NaN      NaN   
junk     NaN       NaN       NaN       NaN       NaN       NaN      NaN   

                cp  trestbps       htn  ...       om2   rcaprox   rcadist  \
id        0.107745  0.

Aqui podemos ver la matriz de correlación. Buscando remover atributos correlacionados podemos realizar lo siguiente:

In [37]:
umbral = 0.35

to_drop = [column for column in upper.columns if any(upper[column] > umbral)]

df = dataset.drop(dataset[to_drop], axis=1)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899 entries, 0 to 898
Data columns (total 33 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   dataset   899 non-null    object 
 1   id        899 non-null    int64  
 2   ccf       899 non-null    int64  
 3   age       899 non-null    int64  
 4   sex       899 non-null    int64  
 5   painloc   617 non-null    float64
 6   painexer  617 non-null    float64
 7   pncaden   0 non-null      float64
 8   trestbps  840 non-null    float64
 9   htn       865 non-null    float64
 10  smoke     230 non-null    float64
 11  fbs       809 non-null    float64
 12  dm        95 non-null     float64
 13  famhist   477 non-null    float64
 14  restecg   897 non-null    float64
 15  ekgmo     846 non-null    float64
 16  ekgday    845 non-null    float64
 17  dig       831 non-null    float64
 18  prop      833 non-null    float64
 19  nitr      834 non-null    float64
 20  diuretic  817 non-null    float6

Esto resulto en una reducción del conjunto de datos de 72 atributos a 32., con un umbral de 0.35

### RapidMiner

En RapidMiner el proceso es muy sencillo gracias al operador `Remove Correlated Attributes`.

![RapidMiner 01](../images/sa-01.png)

## Optimización de Atributos

### Python

Volvamos a cargar los datos e intentemos optimizar la selección de atributos basandonos en pesos.

In [38]:
dataset: pd.DataFrame = pd.read_csv('./datasets/clean.csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 29 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   dataset   572 non-null    object 
 1   id        572 non-null    int64  
 2   age       572 non-null    int64  
 3   sex       572 non-null    int64  
 4   painloc   572 non-null    float64
 5   painexer  572 non-null    float64
 6   trestbps  572 non-null    float64
 7   htn       572 non-null    float64
 8   chol      572 non-null    float64
 9   fbs       572 non-null    float64
 10  restecg   572 non-null    float64
 11  ekgmo     572 non-null    int64  
 12  ekgday    572 non-null    int64  
 13  dig       572 non-null    float64
 14  prop      572 non-null    float64
 15  nitr      572 non-null    float64
 16  pro       572 non-null    float64
 17  diuretic  572 non-null    float64
 18  thaldur   572 non-null    float64
 19  tpeakbpd  572 non-null    float64
 20  oldpeak   572 non-null    float6

Utilizaremos la libreria `sklearn` para realizar la optimización de atributos. En este caso utilizaremos el algoritmo `SelectFrommodel` que utiliza un modelo de aprendizaje para calcular los pesos de los atributos. En este caso utilizaremos un modelo de regresión logística.

In [39]:
pipeline_for_selection = Pipeline([
    ('scaler', StandardScaler()),
    ('regression', LogisticRegression(max_iter=1000))
])

def coef_getter(estimator):
    return estimator.named_steps['regression'].coef_

selector = SelectFromModel(estimator=pipeline_for_selection, threshold=0.2, importance_getter=coef_getter)
selector.fit(dataset.drop(['num', 'dataset', 'id'], axis=1), dataset['num'])

print(np.where(selector.get_support())[0])

[ 1  6 12 15 16 18 20 21 22 23 24 25]


El resultado es una seleccion de 12 atributos, con un umbral de 0.2. Veamos cuales son:

In [40]:
print(dataset.drop(['num', 'dataset', 'id'], axis=1).columns[selector.get_support()])

Index(['sex', 'chol', 'prop', 'diuretic', 'thaldur', 'oldpeak', 'ladprox',
       'laddist', 'cxmain', 'om1', 'rcaprox', 'rcadist'],
      dtype='object')


### RapidMiner

En RapidMiner podemos utilizar el operador `Optimize Selection` en combinacion con el operador `Select By Weights`. El primero requiere un modelo de aprendizaje, en este caso utilizaremos un modelo de regresión logística. El segundo requiere un umbral, en este caso utilizaremos 0.2.

![RapidMiner 02](../images/sa-02.png)
![RapidMiner 03](../images/sa-03.png)
![RapidMiner 04](../images/sa-04.png)

La selección resultante es de 8 atributos:

![RapidMiner 05](../images/sa-05.png)